In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup # library for web scrapping  

!conda install -c conda-forge geocoder --yes
import geocoder

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\MADHUMITHA\Anaconda3

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2




ratelim-0.1.6        | 6 KB      |            |   0% 
ratelim-0.1.6        | 6 KB      | ########## | 100% 

geocoder-1.38.1      | 53 KB     |            |   0% 
geocoder-1.38.1      | 53 KB     | ########## | 100% 
Preparing transaction

In [2]:
CLIENT_ID = 'HR0R1IEP0WDXFYM53HU2GUJLU0GNIEMP4EY5GBSYEN3QGNUW' # your Foursquare ID
CLIENT_SECRET = 'GUVXPO0FGF5DNXWWU5GB50GXLFDRAAH5TOZCER1YOVSN0N0F' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HR0R1IEP0WDXFYM53HU2GUJLU0GNIEMP4EY5GBSYEN3QGNUW
CLIENT_SECRET:GUVXPO0FGF5DNXWWU5GB50GXLFDRAAH5TOZCER1YOVSN0N0F


In [11]:
pwd

'C:\\Users\\MADHUMITHA'

In [13]:
df = pd.read_csv('Crimedata.csv')

In [14]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,...,28.0,25.0,04B,1144606.0,1903566.0,2015,02/10/2018 03:50:01 PM,41.891399,-87.744385,"(41.891398861, -87.744384567)"
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,...,15.0,67.0,26,1166468.0,1860715.0,2015,02/10/2018 03:50:01 PM,41.773372,-87.665319,"(41.773371528, -87.665319468)"
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,4.0,39.0,08B,1185075.0,1875622.0,2015,02/10/2018 03:50:01 PM,41.813861,-87.596643,"(41.81386068, -87.596642837)"
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,...,3.0,40.0,08B,1178033.0,1870804.0,2015,02/10/2018 03:50:01 PM,41.800802,-87.622619,"(41.800802415, -87.622619343)"
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,28.0,25.0,03,1144920.0,1898709.0,2015,02/10/2018 03:50:01 PM,41.878065,-87.743354,"(41.878064761, -87.743354013)"


In [17]:
# Taking only the most recent year (2016) and dropping the rest
df.drop(df.index[df['Year'] != 2015], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

In [18]:
# Shape of the data frame
df.shape

(263770, 22)

In [19]:
# View the information of the dataset 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263770 entries, 0 to 263769
Data columns (total 22 columns):
ID                      263770 non-null int64
Case Number             263770 non-null object
Date                    263770 non-null object
Block                   263770 non-null object
IUCR                    263770 non-null object
Primary Type            263770 non-null object
Description             263770 non-null object
Location Description    263427 non-null object
Arrest                  263770 non-null bool
Domestic                263770 non-null bool
Beat                    263770 non-null int64
District                263770 non-null float64
Ward                    263768 non-null float64
Community Area          263770 non-null float64
FBI Code                263770 non-null object
X Coordinate            257888 non-null float64
Y Coordinate            257888 non-null float64
Year                    263770 non-null int64
Updated On              263770 non-null objec

In [22]:
df['Block'].value_counts()

001XX N STATE ST        768
0000X W TERMINAL ST     507
008XX N MICHIGAN AVE    428
076XX S CICERO AVE      401
0000X N STATE ST        285
                       ... 
044XX W 64TH ST           1
100XX S TORRENCE AVE      1
030XX E 97TH ST           1
083XX S KNOX AVE          1
043XX S OAKLEY AVE        1
Name: Block, Length: 27495, dtype: int64

In [31]:
Crimedata_Latitude = df.loc[1, 'Latitude']
Crimedata_Longitude = df.loc[1, 'Longitude']

Crimedata_name = df.loc[1, 'Location']

print('Latitude and Longitude values of {} are {}, {}.'.format(Crimedata_name, 
                                                               Crimedata_Latitude, 
                                                               Crimedata_Longitude))

Latitude and Longitude values of (41.773371528, -87.665319468) are 41.773371528, -87.66531946799999.
